# Heat Transmission Calculation Notebook

This notebook sets up a Python environment and demonstrates interactive forms and dropdowns for heat transmission calculations using Jupyter widgets.

## Install and Import Jupyter Widgets

The following cell ensures `ipywidgets` is installed and imports the necessary modules for interactive widgets.

In [1]:
# Install ipywidgets if not already installed (uncomment if needed)
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

## Warmtedoorgangscoëfficiënt (U-waarde)

$$U = \frac{1}{R_e + R_c + R_i} \quad \left[\frac{W}{m^2 \cdot K}\right]$$

| Symbool | Omschrijving | Standaardwaarde |
|---------|-------------|-----------------|
| $R_i$ | Warmte-overgangsweerstand binnenzijde | 0,13 m²·K/W |
| $R_e$ | Warmte-overgangsweerstand buitenzijde | 0,04 m²·K/W |
| $R_c$ | Warmteweerstand constructie | $\sum d / \lambda$ |

Voeg lagen toe via de knop hieronder. Per laag kies je een materiaal uit de lijst of vul je de R-waarde handmatig in (voor bijv. luchtspouw).


In [7]:
import json
import ipywidgets as widgets
from IPython.display import display, HTML

# ── Data ─────────────────────────────────────────────────────────────────────
with open('material_properties.json', 'r', encoding='utf-8') as f:
    materials = json.load(f)

# For these categories the deepest value is the U-value [W/(m²·K)], not λ
U_VALUE_CATS = {'glas', 'deuren'}
# For these categories the value is already an R-value [m²·K/W]
R_VALUE_CATS = {'vloeren'}

# Standard surface resistances
SURFACE_R = {
    'Binnenzijde  —  Ri = 0,13 m²·K/W': 0.13,
    'Buitenzijde  —  Re = 0,04 m²·K/W': 0.04,
}

# ── Helpers ───────────────────────────────────────────────────────────────────
def scalar(val):
    """Return the lowest float from a value that may be a list/range."""
    if isinstance(val, list):
        return float(min(val))
    if isinstance(val, (int, float)):
        return float(val)
    return None

def sub_keys(main):
    v = materials.get(main, {})
    return list(v.keys()) if isinstance(v, dict) else []

def third_keys(main, sub):
    v = materials.get(main, {}).get(sub, None)
    return list(v.keys()) if isinstance(v, dict) else []

def raw_value(main, sub, third=None):
    v = materials.get(main, {}).get(sub, None)
    if isinstance(v, dict) and third:
        v = v.get(third, None)
    return v

# ── LayerWidget ───────────────────────────────────────────────────────────────
class LayerWidget:
    def __init__(self, update_cb, remove_cb):
        self.update_cb = update_cb
        self.remove_cb = remove_cb

        # Input mode toggle
        self.mode = widgets.ToggleButtons(
            options=['Materiaallijst', 'Handmatige R'],
            button_style='', layout=widgets.Layout(width='auto')
        )

        # ── Material selectors ──
        self.cat_dd = widgets.Dropdown(
            options=list(materials.keys()), description='Categorie:',
            layout=widgets.Layout(width='220px')
        )
        self.sub_dd = widgets.Dropdown(
            options=sub_keys(self.cat_dd.value), description='Materiaal:',
            layout=widgets.Layout(width='270px')
        )
        self.third_dd = widgets.Dropdown(
            options=[], description='Subtype:',
            layout=widgets.Layout(width='220px', visibility='hidden')
        )
        self.thickness = widgets.BoundedFloatText(
            value=0.10, min=0.0, max=10.0, step=0.001,
            description='Dikte d [m]:', layout=widgets.Layout(width='195px')
        )
        self.lam_lbl = widgets.HTML(value='')

        # ── Manual R ──
        self.manual_r = widgets.BoundedFloatText(
            value=0.10, min=0.0, max=100.0, step=0.01,
            description='R [m²·K/W]:', layout=widgets.Layout(width='195px')
        )

        # Effective-R feedback & remove button
        self.r_lbl   = widgets.HTML(value='')
        self.rem_btn = widgets.Button(
            description='✕ Verwijder', button_style='danger',
            layout=widgets.Layout(width='130px', height='30px')
        )

        # Sub-boxes for each mode
        self.mat_box = widgets.VBox([
            widgets.HBox([self.cat_dd, self.sub_dd, self.third_dd]),
            widgets.HBox([self.thickness, self.lam_lbl]),
        ])
        self.man_box = widgets.VBox([self.manual_r])

        # Wire events
        self.mode.observe(self._on_mode, names='value')
        self.cat_dd.observe(self._on_cat, names='value')
        self.sub_dd.observe(self._on_sub, names='value')
        self.third_dd.observe(self._recalc, names='value')
        self.thickness.observe(self._recalc, names='value')
        self.manual_r.observe(self._recalc, names='value')
        self.rem_btn.on_click(lambda _: self.remove_cb(self))

        self._refresh_sub()
        self._refresh_third()
        self._recalc(None)

        self.box = widgets.VBox(
            [widgets.HBox([self.mode, self.rem_btn]),
             self.mat_box, self.r_lbl],
            layout=widgets.Layout(
                border='1px solid #bbb', padding='6px 10px',
                margin='4px 0', border_radius='4px'
            )
        )

    # ── private ──────────────────────────────────────────
    def _on_mode(self, _):
        inner = self.mat_box if self.mode.value == 'Materiaallijst' else self.man_box
        self.box.children = [self.box.children[0], inner, self.r_lbl]
        self._recalc(None)

    def _on_cat(self, _):
        self._refresh_sub()
        self._refresh_third()
        self._recalc(None)

    def _on_sub(self, _):
        self._refresh_third()
        self._recalc(None)

    def _refresh_sub(self):
        opts = sub_keys(self.cat_dd.value)
        self.sub_dd.options = opts if opts else ['—']

    def _refresh_third(self):
        sub = self.sub_dd.value if self.sub_dd.options else ''
        opts = third_keys(self.cat_dd.value, sub)
        if opts:
            self.third_dd.options = opts
            self.third_dd.layout.visibility = 'visible'
        else:
            self.third_dd.options = ['—']
            self.third_dd.layout.visibility = 'hidden'

    def _recalc(self, _):
        r = self.get_r()
        if r is not None:
            self.r_lbl.value = (
                f'<span style="color:#1a7a1a;font-weight:bold">'
                f'&nbsp;&nbsp;→&nbsp; R = {r:.3f} m²·K/W</span>')
        else:
            self.r_lbl.value = (
                '<span style="color:#c00">'
                '&nbsp;&nbsp;→&nbsp; R kan niet worden bepaald</span>')

        # λ hint label (only relevant for standard materials)
        cat = self.cat_dd.value
        if self.mode.value != 'Materiaallijst' or cat in U_VALUE_CATS or cat in R_VALUE_CATS:
            self.lam_lbl.value = ''
            return
        third = self.third_dd.value if self.third_dd.layout.visibility == 'visible' else None
        val   = raw_value(cat, self.sub_dd.value, third)
        if isinstance(val, list):
            self.lam_lbl.value = (
                f'<span style="color:#555">'
                f'&nbsp; λ = {val[0]} – {val[1]} W/(m·K) '
                f'&nbsp;(laagste waarde {min(val):.4f} gebruikt)</span>')
        elif isinstance(val, (int, float)):
            self.lam_lbl.value = (
                f'<span style="color:#555">'
                f'&nbsp; λ = {val} W/(m·K)</span>')
        else:
            self.lam_lbl.value = ''

        self.update_cb()

    # ── public ───────────────────────────────────────────
    def get_r(self):
        if self.mode.value == 'Handmatige R':
            return self.manual_r.value

        cat   = self.cat_dd.value
        sub   = self.sub_dd.value
        third = self.third_dd.value if self.third_dd.layout.visibility == 'visible' else None
        val   = raw_value(cat, sub, third)

        if cat in U_VALUE_CATS:
            u = scalar(val)
            return (1.0 / u) if u else None

        if cat in R_VALUE_CATS:
            return scalar(val)

        lam = scalar(val)
        d   = self.thickness.value
        return (d / lam) if (lam and lam > 0 and d > 0) else None

    def row_info(self):
        cat   = self.cat_dd.value
        sub   = self.sub_dd.value
        third = self.third_dd.value if self.third_dd.layout.visibility == 'visible' else None
        r     = self.get_r()

        if self.mode.value == 'Handmatige R':
            return {'naam': 'Handmatig', 'd': None, 'lam': '—', 'R': r}

        val = raw_value(cat, sub, third)
        label = f'{cat} / {sub}' + (f' / {third}' if third else '')

        if cat in U_VALUE_CATS:
            u = scalar(val)
            return {'naam': label, 'd': None, 'lam': f'(U={u:.2f})', 'R': r}
        if cat in R_VALUE_CATS:
            return {'naam': label, 'd': None, 'lam': '(R-waarde)', 'R': r}

        lam = scalar(val)
        return {
            'naam': label,
            'd': self.thickness.value,
            'lam': f'{lam:.4f}' if lam else '—',
            'R':   r,
        }


# ── Main UI ───────────────────────────────────────────────────────────────────
layers    = []
layers_box = widgets.VBox([])
out       = widgets.Output()

ri_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Binnenzijde  —  Ri = 0,13 m²·K/W',
    description='Ri (binnen):',
    layout=widgets.Layout(width='340px')
)
re_dd = widgets.Dropdown(
    options=list(SURFACE_R.keys()),
    value='Buitenzijde  —  Re = 0,04 m²·K/W',
    description='Re (buiten):',
    layout=widgets.Layout(width='340px')
)
ri_dd.observe(lambda _: refresh(), names='value')
re_dd.observe(lambda _: refresh(), names='value')

add_btn = widgets.Button(
    description='＋ Voeg laag toe', button_style='primary',
    layout=widgets.Layout(width='160px')
)


def refresh(*_):
    ri = SURFACE_R[ri_dd.value]
    re = SURFACE_R[re_dd.value]

    with out:
        out.clear_output(wait=True)

        rows_html = ''
        total_d  = 0.0
        total_rc = 0.0

        # Inside surface
        rows_html += (
            f'<tr class="surface"><td>lucht (binnen)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{ri:.2f}</td></tr>\n'
        )

        for i, layer in enumerate(layers):
            info = layer.row_info()
            r    = info['R']
            d    = info['d']
            d_str = f'{d:.3f}' if isinstance(d, float) else '—'
            r_str = f'{r:.3f}' if r is not None else '?'
            if isinstance(d, float): total_d  += d
            if r is not None:        total_rc += r
            rows_html += (
                f'<tr><td>{info["naam"]}</td>'
                f'<td>{d_str}</td><td>{info["lam"]}</td>'
                f'<td>{r_str}</td><td>—</td></tr>\n'
            )

        # Outside surface
        rows_html += (
            f'<tr class="surface"><td>lucht (buiten)</td>'
            f'<td>—</td><td>—</td><td>—</td><td>{re:.2f}</td></tr>\n'
        )

        total_r  = ri + total_rc + re
        u        = 1.0 / total_r if total_r > 0 else None
        u_str    = f'{u:.3f}' if u is not None else '?'
        d_tot    = f'{total_d:.3f}' if total_d > 0 else '—'

        html = f"""
<style>
  .utbl {{ border-collapse:collapse; font-family:sans-serif;
           font-size:13px; min-width:680px; }}
  .utbl th {{ background:#2c5f8a; color:white;
              padding:6px 12px; text-align:left; }}
  .utbl td {{ padding:5px 12px; border-bottom:1px solid #ddd; }}
  .utbl tr:not(.total-row):not(.u-row):hover td {{ background:#f0f7ff; }}
  .utbl .surface td  {{ color:#555; font-style:italic; }}
  .utbl .total-row td {{ font-weight:bold; background:#e8f0fe;
                         border-top:2px solid #2c5f8a; }}
  .utbl .u-row td   {{ font-weight:bold; background:#2c5f8a;
                       color:white; font-size:15px; }}
</style>
<table class="utbl">
  <tr>
    <th>Materiaal / Laag</th>
    <th>d [m]</th>
    <th>λ [W/(m·K)]</th>
    <th>R = d/λ [m²·K/W]</th>
    <th>Ri &amp; Re [m²·K/W]</th>
  </tr>
  {rows_html}
  <tr class="total-row">
    <td>TOTAAL</td>
    <td>{d_tot}</td>
    <td>—</td>
    <td>{total_rc:.3f}</td>
    <td>{ri + re:.2f}</td>
  </tr>
  <tr class="u-row">
    <td colspan="5">
      U = 1 / (R<sub>i</sub> {ri:.2f} + R<sub>c</sub> {total_rc:.3f}
              + R<sub>e</sub> {re:.2f})
      &nbsp;=&nbsp; <b>{u_str} W/(m²·K)</b>
    </td>
  </tr>
</table>"""
        display(HTML(html))


def add_layer(_=None):
    layer = LayerWidget(update_cb=refresh, remove_cb=remove_layer)
    layers.append(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


def remove_layer(layer):
    layers.remove(layer)
    layers_box.children = [l.box for l in layers]
    refresh()


add_btn.on_click(add_layer)

display(widgets.VBox([
    widgets.HTML('<b>Overgangsweerstanden</b>'),
    widgets.HBox([ri_dd, re_dd]),
    widgets.HTML('<b style="margin-top:10px;display:block">Constructielagen (Rc)</b>'),
    layers_box,
    add_btn,
    widgets.HTML('<b style="margin-top:10px;display:block">Resultaat</b>'),
    out,
]))

# Start with one empty layer
add_layer()
